# Sistemas de Recomendación

<div align="center"><a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/MachineLearning/11_Recomendacion/sistemas_recomendacion_sol.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg'/> </a> <br> Recordá abrir en una nueva pestaña </div>

In [ ]:
# 1. Downgrade a NumPy < 2.0 compatible con Surprise
!pip install numpy==1.24.4 --force-reinstall --no-cache-dir

# 2. Reinstalar Surprise compatible
!pip install scikit-surprise --no-binary :all: --no-cache-dir



In [1]:
import pandas as pd
import numpy as np

## Similitud coseno

$$sim(\pmb x, \pmb y) = \frac {\pmb x \cdot \pmb y}{||\pmb x|| \cdot ||\pmb y||}$$

¿Cómo calcularla en Python?

Supongamos que tenemos la siguiente matriz:

|  	| Libro A 	| Libro B 	| Libro C 	|
|-------	|---------	|---------	|---------	|
| Juan 	| 5 	| 4 	| 4 	|
| Diego 	| 4 	| 5 	| 5 	|


Podemos calcular la similitud coseno empleando sklearn:

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
Juan = [5,4,4]
Diego = [4,5,5]
cosine_similarity([Juan, Diego])

array([[1.        , 0.97823198],
       [0.97823198, 1.        ]])

También podemos calcular la similitud a mano:

In [3]:
(5*4 + 4*5 + 4*5)/(np.sqrt(5**2+4**2+4**2)*np.sqrt(4**2+5**2+5**2))

np.float64(0.9782319760890369)

O empleando Numpy

Calcular la similitud coseno usando numpy (con np.dot y np.linalg.norm)

In [4]:
np.dot(Juan,Diego)/np.dot(np.linalg.norm(Juan), np.linalg.norm(Diego))

np.float64(0.9782319760890369)

Ahora bien, cuando tenemos una matriz user-item de la vida real, tenemos muchos casos faltantes. En esta situación, no podremos calcular la similitud coseno tan fácilmente...

In [5]:
user_item = np.array([[5, np.nan, 4],[4,3,5],[4,5,5],[np.nan, 5, np.nan], [np.nan, 5, 3]])
user_item

array([[ 5., nan,  4.],
       [ 4.,  3.,  5.],
       [ 4.,  5.,  5.],
       [nan,  5., nan],
       [nan,  5.,  3.]])

## Filtrado colaborativo basado en modelos- SVD modelo de factorización de matrices (Surprise)

En esta notebook vamos a emplear la librería surprise. Esta es una librería que se basa en la API de scikit-learn y permite implementar varios algoritmos básicos de recomendación.

Comencemos cargando un dataset clásico en sistemas de recomendación: MovieLens (https://movielens.org/). Esta es una página de recomendación de películas que abrió información histórica.

In [6]:
!pip install surprise
# Bajamos el dataset. En windows pueden descargarlo entrando al link manualmente
!wget https://files.grouplens.org/datasets/movielens/ml-100k/u.data .

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2469555 sha256=3c8e45c795ecc87020c74b136fc5fc808bf06fbdd1bd43f95ec558c7631c07c5
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise
--2025-06-16 23:09:58--  https://files.grouplens.org/datasets/movielens/ml-100k/u.data
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1979173 (1.9M)
Saving to: ‘u.data’

u.data              100%[===================>]   1.89M  12.0MB/s    in 0.2s    

2025-06-16 23:09:58 (12.0 MB/s) - ‘u.data

In [1]:
import pandas as pd

In [2]:
mlens = pd.read_csv("u.data",sep="\t",header=None)
mlens.columns = ["user_id","item_id","rating","timestamp"]

In [3]:
mlens = mlens.drop("timestamp", axis=1)

Dataset con puntuaciones de usuarios a películas.

In [4]:
mlens

,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


El paquete surprise no recibe directamente un objeto DataFrame sino que tiene para parsear y leer un conjunto de datos debe hacerlo a través de dos nuevos objetos: Reader y Dataset. En Reader debemos especificar el valor mínimo y el valor máximo de los ratings y Dataset nos permite leer datos desde distintas fuentes.

In [5]:
from surprise import Dataset, Reader
reader = Reader(rating_scale=(mlens["rating"].min(),mlens["rating"].max()))

Reader define el rango de calificaciones.



Dataset.load_from_df convierte el DataFrame en un formato que Surprise puede usar internamente. Surprise requiere este paso porque no trabaja directamente con pandas.

In [6]:
dataset = Dataset.load_from_df(mlens,reader)

In [7]:
dataset

Ahora cargue SVD y GridSearchCV, ambos de surprise.


In [8]:
from surprise import SVD
from surprise.model_selection import GridSearchCV

SVD: modelo de factorización de matrices (filtrado colaborativo basado en modelos).

GridSearchCV: búsqueda de los mejores hiperparámetros para ese modelo.

Genere una grilla de parámetros donde se prueben distintas combinaciones de:  
  - epochs: es la cantidad de pasadas sobre el dataset que hará el algoritmo empleando descenso por el gradiente  
  - biased: usar parámetros de sesgo o no  
  - lr_all: learning rate para todos los parámetros  
  - reg_all: término de regularización para todos los parámetros (lambda)  

In [9]:
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005], 'reg_all': [0.4, 0.6]}

Se prueban varias combinaciones de:

n_epochs: cuántas veces pasa sobre el dataset (SGD)

lr_all: tasa de aprendizaje

reg_all: regularización (lambda)

Se evalúa con:

**RMSE (Root Mean Square Error)**

**FCP (Fraction of Concordant Pairs)**

Esto permite encontrar el modelo que predice mejor las calificaciones y respeta mejor el orden de preferencias del usuario.



Emplee GridSearchCV, SVD y el diccionario con los parámetros para probar, y entrene un modelo. Note que a GridSearchCV necesita pasarle un modelo sin instanciar. Además, setee el parámetro refit a True y con measures = ["rmse","fcp"]

In [10]:
gs = GridSearchCV(SVD, param_grid, measures=['fcp',"rmse"], cv=3, refit=True)

In [11]:
gs.fit(dataset)

Imprima el rmse y el fcp, y la mejor combinación de parámetros

In [12]:
gs.best_score

{'fcp': 0.6974812864666816, 'rmse': 0.9642310713267767}

In [13]:
gs.best_params

{'fcp': {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.6},
 'rmse': {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}}

Guarde el modelo con mayor fcp y prediga el rating para el user id 196 e item id 242

In [14]:
best_model = gs.best_estimator["fcp"]

In [15]:
pred = best_model.predict("196", "242")

In [16]:
pred.est

3.52986

Pruebe empleando otros modelos como SVDpp, NMF, KNNWithZScore e intente superar el valor obtenido

In [17]:
from surprise import SVDpp
gs = GridSearchCV(SVDpp, param_grid, measures=['fcp',"rmse"], cv=3, refit=True)
gs.fit(dataset)
gs.best_score

{'fcp': 0.6985485933656727, 'rmse': 0.9635880974378654}

SVDpp: una versión mejorada de SVD que considera implícitamente los ítems no calificados. Considera no solo los ítems que el usuario calificó, sino también aquellos con los que interactuó (por ejemplo, que vio o visitó).

NMF: factorización no negativa (otro tipo de descomposición de matriz). Todos los valores deben ser ≥ 0.Tiene sentido en contextos donde los componentes deben ser interpretables como "presencias" o "pesos".

KNNWithZScore: filtrado colaborativo basado en memoria, pero con normalización (Z-score).Trabaja con los vecinos más cercanos (K-Nearest Neighbors), usando la puntuación de usuarios similares para hacer predicciones. Es una mejora sobre KNNBasic que normaliza las puntuaciones del usuario usando Z-score.